In [103]:
!pip install python-dotenv



In [104]:
from dotenv import load_dotenv, dotenv_values
import os

load_dotenv("../key.env")
jinaai_api_key = os.getenv("JINAAI_API_KEY")

#print(os.getenv("JINAAI_API_KEY"))


In [105]:
from llama_index.embeddings.jinaai import JinaEmbedding

text_embed_model = JinaEmbedding(
    api_key=jinaai_api_key,
    model="jina-embeddings-v3",
    # choose `retrieval.passage` to get passage embeddings
    task="retrieval.passage",
)



In [106]:
gesetze = ['../Gesetze/AGG.txt','../Gesetze/ArbSchG.txt','../Gesetze/ArbzG.txt','../Gesetze/BetrVG.txt',
           '../Gesetze/BUrlG.txt','../Gesetze/bgb_611_630.txt','../Gesetze/entgfg.txt','../Gesetze/KSchG.txt',
           '../Gesetze/MiLoG.txt','../Gesetze/NachwG.txt','../Gesetze/TzBfG.txt'
           ]


In [107]:
documents = []
for gesetz in gesetze:
    with open(gesetz, 'r', encoding='utf-8') as f:
        documents.append(f.read())
        print(gesetz)


../Gesetze/AGG.txt
../Gesetze/ArbSchG.txt
../Gesetze/ArbzG.txt
../Gesetze/BetrVG.txt
../Gesetze/BUrlG.txt
../Gesetze/bgb_611_630.txt
../Gesetze/entgfg.txt
../Gesetze/KSchG.txt
../Gesetze/MiLoG.txt
../Gesetze/NachwG.txt
../Gesetze/TzBfG.txt


In [108]:
import re

def smart_law_chunking(text, law_name, max_chars=1000):
    """
    Zerlegt Gesetzestexte intelligent in Paragraphen und Absätze.
    Ergebnis: Kleine, sinnvolle Chunks mit Metadaten für die EDA.
    """
    chunks_data = []
    
    # 1. Split nach Paragraphen (z.B. "§ 1", "§ 2a")
    # Das Pattern sucht nach § gefolgt von Zahl und optional Buchstabe
    para_splits = re.split(r'(§\s*\d+[a-z]?)', text)
    
    current_para = "Präambel/Allgemein" # Fallback für Text vor dem ersten §
    
    # re.split mit capture group liefert [text, separator, text, separator...]
    # Wir iterieren und setzen es wieder zusammen
    for i in range(len(para_splits)):
        segment = para_splits[i].strip()
        
        # Wenn das Segment ein Paragraph-Header ist (z.B. "§ 1"), speichern wir ihn als Kontext
        if re.match(r'§\s*\d+[a-z]?', segment):
            current_para = segment
            continue
            
        if not segment: continue

        # 2. Innerhalb des Paragraphen nach Absätzen splitten (z.B. "(1)", "(2)")
        # Pattern: Neue Zeile oder Anfang, gefolgt von (Zahl)
        absatz_splits = re.split(r'(\(\d+\))', segment)
        
        current_absatz = ""
        
        for j in range(len(absatz_splits)):
            part = absatz_splits[j].strip()
            
            # Ist es eine Absatz-Nummer? (z.B. "(1)")
            if re.match(r'\(\d+\)', part):
                current_absatz = part
                continue
            
            if not part: continue
            
            # Hier haben wir den reinen Textinhalt eines Absatzes
            # Konstruiere den vollen Kontext für die EDA (Tooltip)
            full_label = f"{law_name} {current_para} {current_absatz}".strip()
            
            # Falls der Absatz immer noch riesig ist (sehr selten), 
            # splitten wir ihn hart, aber versuchen Satzenden zu respektieren
            if len(part) > max_chars:
                # Hier könnte man noch feiner nach Sätzen splitten, 
                # aber für die EDA ist Absatz-Level meist perfekt.
                sub_parts = [part[k:k+max_chars] for k in range(0, len(part), max_chars)]
                for idx, sub in enumerate(sub_parts):
                    chunks_data.append({
                        "text": sub,
                        "label": f"{full_label} (Teil {idx+1})", # Wichtig für Tooltip
                        "law": law_name,
                        "paragraph": current_para
                    })
            else:
                chunks_data.append({
                    "text": part,
                    "label": full_label, # Das hier zeigst du in der EDA beim Hovern!
                    "gesetz": law_name,
                
                    "paragraph": current_para
                })
                
    return chunks_data

In [109]:
all_chunks_text = [] # Nur der Text für Jina
all_metadatas = []   # Metadaten für Chroma/EDA

print("Starte intelligentes Chunking...")

for idx, doc_text in enumerate(documents):
    law_name = document_names[idx] # z.B. "KSchG"
    
    # Nutze die neue Funktion
    structured_chunks = smart_law_chunking(doc_text, law_name)
    
    for chunk in structured_chunks:
        # Wir fügen Kontext (Label) zum Text hinzu, damit das Embedding besser wird
        # Jina "sieht" dann: "KSchG § 1 (1): Die Kündigung..." statt nur "Die Kündigung..."
        embedding_input_text = f"{chunk['label']}: {chunk['text']}"
        
        all_chunks_text.append(embedding_input_text)
        all_metadatas.append(chunk) # Das hier kommt in die Chroma Metadaten!

Starte intelligentes Chunking...


In [110]:
print(f"Erstelle Embeddings für {len(all_chunks_text)} Chunks...")
embeddings = text_embed_model.get_text_embedding_batch(all_chunks_text)

print(f"Fertig! Erstes Label: {all_metadatas[0]['label']}")

Erstelle Embeddings für 1675 Chunks...


KeyboardInterrupt: 

In [111]:
import chromadb 

from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from IPython.display import Markdown, display

In [112]:
# ... (dein Code davor mit smart_law_chunking) ...

print(f"Anzahl Chunks: {len(all_chunks_text)}")

# 1. IDs NEU GENERIEREN (Der Fix)
# Wir erstellen eine ID für jeden Chunk basierend auf dem Gesetz und dem Index
ids = [f"{meta['law']}_{i}" for i, meta in enumerate(all_metadatas)]

# Check zur Sicherheit (muss jetzt gleich sein)
print(f"Anzahl IDs: {len(ids)}") 

# 2. Chroma Client & Collection (wie gehabt)
chroma_client = chromadb.PersistentClient(path="../VektorDB2")
chroma_collection = chroma_client.get_or_create_collection(
    "VektorDB2",
    metadata={"description": "Deutsche Arbeitsgesetze Embeddings"}
)

# 3. Add (jetzt sollte es funktionieren)
chroma_collection.add(
    ids=ids,
    embeddings=embeddings,      
    documents=all_chunks_text, 
    metadatas=all_metadatas     
)

print("Erfolgreich gespeichert!")

Anzahl Chunks: 1675
Anzahl IDs: 1675
Erfolgreich gespeichert!
